https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/

In [1]:
import copy
import numpy as np

In [2]:
np.random.seed(42)

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def sigmoid_output_to_derivative(output):
    return output * (1 - output)

In [5]:
int2binary = {}
binary_dim = 8

In [6]:
largest_number = pow(2, binary_dim)
binary = np.unpackbits(np.array([range(largest_number)], dtype = np.uint8).T, axis = 1)

In [7]:
for i in range(largest_number):
    int2binary[i] = binary[i]

In [8]:
alpha, input_dim, hidden_dim, output_dim = 0.1, 2, 32, 1

In [9]:
synapse_0 = 2 * np.random.random((input_dim, hidden_dim)) - 1
synapse_1 = 2 * np.random.random((hidden_dim, output_dim)) - 1
synapse_h = 2 * np.random.random((hidden_dim, hidden_dim)) - 1

synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [10]:
for iteration in range(100000):
    a_int = np.random.randint(largest_number / 2)
    a = int2binary[a_int]
    
    b_int = np.random.randint(largest_number / 2)
    b = int2binary[b_int]
    
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    d = np.zeros_like(c)
    
    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    
    for position in range(binary_dim):
        x = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T
        
        layer_1 = sigmoid(np.dot(x, synapse_0) + np.dot(layer_1_values[-1], synapse_h))
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))
        
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error) * sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
        
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        layer_1_values.append(copy.deepcopy(layer_1))
        
    future_layer_1_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        x = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position - 1]
        prev_layer_1 = layer_1_values[-position - 2]
        
        layer_2_delta = layer_2_deltas[-position - 1]
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += x.T.dot(layer_1_delta)
        
        future_layer_1_delt1 = layer_1_delta
        
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha
    
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    if iteration % 1000 == 0:
        print("Error: " + str(overallError))
        print("Pred: " + str(d))
        print("True: " + str(c))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x*pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

Error: [ 4.07974037]
Pred: [1 1 1 1 1 1 1 1]
True: [0 1 0 0 1 0 1 0]
8 + 66 = 255
------------
Error: [ 3.7863518]
Pred: [0 0 0 1 0 0 1 0]
True: [0 0 0 1 0 1 0 1]
11 + 10 = 18
------------
Error: [ 3.60777713]
Pred: [1 1 1 1 1 1 1 0]
True: [0 1 1 0 1 1 1 0]
68 + 42 = 254
------------
Error: [ 4.03256585]
Pred: [0 1 0 1 0 1 1 0]
True: [0 0 1 1 1 0 0 0]
35 + 21 = 86
------------
Error: [ 3.04073502]
Pred: [0 1 0 0 0 0 0 0]
True: [0 1 0 0 1 0 1 1]
64 + 11 = 64
------------
Error: [ 3.01042117]
Pred: [0 1 1 0 1 1 1 1]
True: [0 1 1 0 1 1 1 1]
82 + 29 = 111
------------
Error: [ 1.13093815]
Pred: [1 0 1 0 1 0 1 0]
True: [1 0 1 0 1 0 1 0]
62 + 108 = 170
------------
Error: [ 0.89185978]
Pred: [0 0 1 0 0 1 1 1]
True: [0 0 1 0 0 1 1 1]
17 + 22 = 39
------------
Error: [ 0.96724288]
Pred: [0 1 1 1 1 1 0 1]
True: [0 1 1 1 1 1 0 1]
1 + 124 = 125
------------
Error: [ 0.33734575]
Pred: [0 0 0 1 1 0 1 1]
True: [0 0 0 1 1 0 1 1]
26 + 1 = 27
------------
Error: [ 0.43293759]
Pred: [0 1 1 1 0 1 1 1]
Tr